<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/AC_OptimalPowerFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AC Optimal Power Flow 

This Jupyter Notebook solves an instance of the AC Optimal Power Flow Problem described in XXX

## Requirements

In [1]:
!pip install -q pyomo
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
import pyomo.environ as pe
import pandas as pd
import numpy as np
ipopt = pe.SolverFactory('ipopt', executable='/content/ipopt')

     |████████████████████████████████| 9.6 MB 4.7 MB/s 
     |████████████████████████████████| 49 kB 2.9 MB/s 


## Input Data

In [2]:
Sbase = 100
gen = pd.DataFrame({
       'unit':   [1,    2], 
       'bus':    [1,    2],
       'q_cost': [0.01, 0.02],
       'l_cost': [10,   20], 
       'pmin':   [0,    0], 
       'pmax':   [100,  100],
       'qmin':   [-50,  -50], 
       'qmax':   [50,   50]})
lin = pd.DataFrame({
       'line': [1,  2,   3], 
       'from': [1,  1,   2],
       'to':   [2,  3,   3], 
       'r':    [0.1,  0.1,   0.1], 
       'x':    [0.01, 0.01,  0.01], 
       'cap':  [25, 100, 100]})
dem = pd.DataFrame({
       'dem':    [1  ], 
       'bus':    [3  ],
       'level':  [150],
       'factor': [0.1]})
ngen = len(gen)
nbus = max(max(lin['from']),max(lin['to']))
nlin = len(lin)
ndem = len(dem)

## Determining the generator-bus and demand-bus incidence matrixes

In [3]:
gen_bus = np.zeros((ngen,nbus))
for g in range(ngen):
  gen_bus[g,gen.loc[g,'bus']-1]=1
dem_bus = np.zeros((ndem,nbus))
for g in range(ndem):
  dem_bus[g,dem.loc[g,'bus']-1]=1

## Determining the admitance and capacity matrices

In [4]:
Y = np.zeros((nbus,nbus),dtype = 'complex_')
S = np.zeros((nbus,nbus))
for l in range(nlin):  
  bus_from = lin.loc[l,'from'] - 1
  bus_to = lin.loc[l,'to'] - 1  
  y = 1/(lin.loc[l,'r'] + lin.at[l,'x']*1j)
  Y[bus_from,bus_from] = Y[bus_from,bus_from] + y
  Y[bus_to,bus_to] = Y[bus_to,bus_to] + y
  Y[bus_from,bus_to] = Y[bus_from,bus_to] - y
  Y[bus_to,bus_from] = Y[bus_to,bus_from] - y  
  S[bus_from,bus_to] = lin.loc[l,'cap']
  S[bus_to,bus_from] = lin.loc[l,'cap']
G = np.real(Y)
B = np.imag(Y)
print(G,B)

[[19.8019802 -9.9009901 -9.9009901]
 [-9.9009901 19.8019802 -9.9009901]
 [-9.9009901 -9.9009901 19.8019802]] [[-1.98019802  0.99009901  0.99009901]
 [ 0.99009901 -1.98019802  0.99009901]
 [ 0.99009901  0.99009901 -1.98019802]]




## Solving the AC Optimal Power Flow

In [5]:
# Model
m = pe.ConcreteModel()
# Sets
m.g = pe.Set(initialize=list(range(ngen)))
m.j = pe.Set(initialize=list(range(ndem)))
m.n = pe.Set(initialize=list(range(nbus)))
m.l = pe.Set(initialize=list(range(nlin)))
# Variables
m.p = pe.Var(m.g,within=pe.NonNegativeReals)
m.q = pe.Var(m.g)
m.v = pe.Var(m.n,bounds=(0.95,1.05),initialize=1.0)
m.ang = pe.Var(m.n,bounds=(-1.57,1.57),initialize=0.0)
m.fp = pe.Var(m.n,m.n)
m.fq = pe.Var(m.n,m.n)
def obj_rule(m):
  return sum(gen.loc[g,'q_cost']*m.p[g]*m.p[g] + gen.loc[g,'l_cost']*m.p[g] for g in m.g)
m.obj = pe.Objective(rule=obj_rule)
# Active power balance
def act_bal_rule(m,n):
  return sum(gen_bus[g,n]*m.p[g] for g in m.g) == sum(dem_bus[j,n]*dem.loc[j,'level'] for j in m.j) + sum(m.fp[n,j] for j in m.n)
m.act_bal = pe.Constraint(m.n,rule=act_bal_rule)
# Reactive power balance
def react_bal_rule(m,n):
  return sum(gen_bus[g,n]*m.q[g] for g in m.g) == sum(dem_bus[j,n]*dem.loc[j,'level']*dem.loc[j,'factor'] for j in m.j) + sum(m.fq[n,j] for j in m.n)
m.react_bal = pe.Constraint(m.n,rule=react_bal_rule)
# Active power flow
def act_flow_rule(m,i,j):
  return m.fp[i,j] == -G[i,j]*m.v[i]*m.v[i]*Sbase + m.v[i]*m.v[j]*(G[i,j]*pe.cos(m.ang[i]-m.ang[j]) + B[i,j]*pe.sin(m.ang[i]-m.ang[j]))*Sbase
m.act_flow = pe.Constraint(m.n,m.n,rule=act_flow_rule)
# Reactive power flow
def react_flow_rule(m,i,j):
  return m.fq[i,j] == B[i,j]*m.v[i]*m.v[i]*Sbase + m.v[i]*m.v[j]*(G[i,j]*pe.sin(m.ang[i]-m.ang[j]) - B[i,j]*pe.cos(m.ang[i]-m.ang[j]))*Sbase
m.react_flow = pe.Constraint(m.n,m.n,rule=react_flow_rule)
# Maximum active generation
def maxp_rule(m,g):
  return m.p[g] <= gen.loc[g,'pmax']
m.maxp = pe.Constraint(m.g, rule=maxp_rule)
# Maximum reactive generation
def maxq_rule(m,g):
  return m.q[g] <= gen.loc[g,'qmax']
m.maxq = pe.Constraint(m.g, rule=maxq_rule)
# Minimum reactive generation
def minq_rule(m,g):
  return m.q[g] >= gen.loc[g,'qmin']
m.minq = pe.Constraint(m.g, rule=minq_rule)
# Maximum power flow
def max_flow_rule(m,i,j):
  return m.fp[i,j]*m.fp[i,j] + m.fq[i,j]*m.fq[i,j] <= S[i,j]*S[i,j]
m.max_flow = pe.Constraint(m.n,m.n, rule=max_flow_rule)
# We solve the optimization problem using IPOPT
ipopt.solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 28
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.1293962001800537
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


## Solution Output

In [6]:
# Print results
print('obj =',m.obj())
print('p1 =',m.p[0].value)
print('p2 =',m.p[1].value)
print('v1 =',m.v[0].value)
print('v2 =',m.v[1].value)
print('v3 =',m.v[2].value)

obj = 2425.645237082433
p1 = 100.00000099974473
p2 = 62.38977698236552
v1 = 1.05
v2 = 1.0383176113238324
v3 = 0.9657196040112259
